# script to capture images in a loop. 

## make sure, in the web interface, you close the camera. Run the cells one by one.

In [1]:
#some imports
import time
from PIL import Image
import numpy as np
import pandas as pd
import pathlib
from picamera2 import Picamera2, Preview
import pprint
from ipywidgets import interact, interactive
from IPython.display import display
import matplotlib.pyplot as plt
from characterization_ams.stats_engine import stats
from characterization_ams.emva import emva
from characterization_ams.standard_tests import ptc


In [2]:
amount = 20 #numbers of pictures to capture per setting
bit_mode = 10
analog_gain = 1
exposure = 200 #in us #np.arange(1000, 10000, 500, dtype=int) #start, stop, step - this must be an array type. can also be, [100,200,300,400] etc..
min_exposure = 100
#select mode a few cells below.
TIFF = True
board_id = 'poncha'
UPLOAD_FOLDER = pathlib.Path(f"./images{board_id}")
UPLOAD_FOLDER.mkdir(parents=False, exist_ok=True)

temp_save_dir = UPLOAD_FOLDER
final_save_dir = UPLOAD_FOLDER

In [3]:
#view camera model
pprint.pprint(Picamera2.global_camera_info() ) #before init ;
#print all sensor modes
with Picamera2() as picam2:
    modes = picam2.sensor_modes
    pprint.pprint(picam2.sensor_modes)

[{'Id': '/base/soc/i2c0mux/i2c@1/poncha110@36',
  'Location': 2,
  'Model': 'poncha110color',
  'Num': 0,
  'Rotation': 0}]
[{'bit_depth': 10,
  'crop_limits': (0, 0, 1080, 1082),
  'exposure_limits': (33, 69287, None),
  'format': SBGGR10_CSI2P,
  'fps': 27.34,
  'size': (1080, 1082),
  'unpacked': 'SBGGR10'}]


[0:08:06.485957864] [2283]  INFO Camera camera_manager.cpp:313 libcamera v0.3.0+65-6ddd79b5-dirty (2024-09-17T09:54:50+02:00)
[0:08:06.508670337] [2309]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[0:08:06.509854646] [2309]  WARN RPiAlsc alsc.cpp:170 no luminance table - assume unity everywhere
[0:08:06.510756771] [2309]  WARN RPI vc4.cpp:392 Mismatch between Unicam and CamHelper for embedded data usage!
[0:08:06.512315485] [2309]  INFO RPI vc4.cpp:446 Registered camera /base/soc/i2c0mux/i2c@1/poncha110@36 to Unicam device /dev/media4 and ISP device /dev/media0
[0:08:06.512470243] [2309]  INFO RPI pipeline_base.cpp:1104 Using configuration file '/usr/local/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[0:08:06.537742943] [2283]  INFO Camera camera_manager.cpp:313 libcamera v0.3.0+65-6ddd79b5-dirty (2024-09-17T09:54:50+02:00)
[0:08:06.578271915] [2312]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN insid

# select mode here in the dropdown

In [6]:
for mode in modes:
    if mode['bit_depth']==bit_mode:
        break
selected_mode=mode

In [ ]:
with Picamera2() as picam2:
    preview_config = picam2.create_preview_configuration(main={"size": selected_mode["size"]},
        raw={"format": selected_mode["unpacked"],
            "size": selected_mode["size"],
        })
    picam2.configure(preview_config)

    picam2.start()



    picam2.set_controls({"ExposureTime": exposure , "AnalogueGain": analog_gain})
    
    time.sleep(2)
    #raw = picam2.capture_buffer()
    #np.from_buffer
    
    size = selected_mode["size"]

    width = size[0]
    height = size[1]
    im_stack = []
    for i in range(amount):
        if bit_mode == 8:
            image = picam2.capture_array("raw").view(np.uint8)
        else:
            image = picam2.capture_array("raw").view(np.uint16)
        im_stack.append(image[0:height, 0:width])
        if TIFF:
            # save images    
            pilim = Image.fromarray(image)
            filename = str(f"{UPLOAD_FOLDER}/img_exposure{exposure}_gain{analog_gain}{i}.tiff")
            pilim.save(filename)
        
    # get & save images
    name = f'bm={bit_mode}_ag={analog_gain}_fc={amount}_exp={round(exposure, 1)}_dark.npz'
    temp_im_dir = UPLOAD_FOLDER
    temp_im_path = pathlib.Path(temp_im_dir / name)
    np.savez(str(temp_im_path), im_stack)



[0:09:30.640175415] [2283]  INFO Camera camera_manager.cpp:313 libcamera v0.3.0+65-6ddd79b5-dirty (2024-09-17T09:54:50+02:00)
[0:09:30.654250028] [2384]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[0:09:30.654819020] [2384]  WARN RPiAlsc alsc.cpp:170 no luminance table - assume unity everywhere
[0:09:30.655354697] [2384]  WARN RPI vc4.cpp:392 Mismatch between Unicam and CamHelper for embedded data usage!
[0:09:30.656610698] [2384]  INFO RPI vc4.cpp:446 Registered camera /base/soc/i2c0mux/i2c@1/poncha110@36 to Unicam device /dev/media4 and ISP device /dev/media0
[0:09:30.656692234] [2384]  INFO RPI pipeline_base.cpp:1104 Using configuration file '/usr/local/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[0:09:30.671019120] [2283]  INFO Camera camera_manager.cpp:313 libcamera v0.3.0+65-6ddd79b5-dirty (2024-09-17T09:54:50+02:00)
[0:09:30.685164973] [2388]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN insid

In [ ]:
print(im_stack[0])

In [ ]:
plt.imshow(im_stack[0])
plt.show()

In [ ]:
results = stats.agg_results(im_stack, rename=True)

In [ ]:
results.T

In [ ]:
results.to_csv(board_id+'.csv')